In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

# The 3D-RISM density slices are saved as pickled files in the folder MUP1.
# They are indexed by a number (see d_list) which represents the distance in Angstrom
# from the central slice.

d_list = [-1.0, -0.5, 0.0, 0.5, 1.0, 1.5]

# import slices
path = "../MUP1/MUP1_logfilter8_slices/"
basename = "_density_slice_MUP1_logfilter8.p"
densities = []
for d in d_list:
    filename = path + f"d{d}" + basename
    with open(filename, 'rb') as file_in:
         densities.append(pickle.load(file_in))
            
# import slice coordinates (these are 3D coordinates in
# angstroms, they are needed at the very end to map
# excited qubits to positions in the protein cavity)
path = "../MUP1/MUP1_logfilter8_points/"
basename = "_plane_points_MUP1.p"
points = []
for d in d_list:
    filename = path + f"d{d}" + basename
    with open(filename, 'rb') as file_in:
         points.append(pickle.load(file_in))


# The register associated to each slide can be found in the folder nb/registers.
# Two types of files are saved there:
# - position_<#>.npy: the positions of the qubits in micrometers, as if they were in the QPU
# - rescaled_position_<#>.npy: the positions of the qubits on the same scale as the density slices

# import registers
path = "registers/"
basename = "position_"
positions = []
for i in range(len(d_list)):
    with open(f'registers/position_{i}.npy', 'rb') as file_in:
        pos = np.load(file_in)
    positions.append(pos)

basename = "rescaled_position_"
rescaled_positions = []
for i in range(len(d_list)):
    with open(f'registers/rescaled_position_{i}.npy', 'rb') as file_in:
        res_pos = np.load(file_in)
    rescaled_positions.append(res_pos)

In [5]:
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.circuit import ClassicalRegister
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.utils import algorithm_globals

import sys
sys.path.append('../utils/')
import qubo_utils as qutils 

# for reproducable results
algorithm_globals.random_seed = 3

qubo_matrices = qutils.get_qubo_matrices(densities=densities, rescaled_positions=rescaled_positions)
qubo = qubo_matrices[0] # use the first slice as an example to solve

# get the corresponding Ising Hamiltonian
ising_ham = qutils.get_Ising_hamiltonian(qubo=qubo)

# define the ansatz to be used in vqe
num_qubits = len(qubo)
ansatz = TwoLocal(num_qubits, 'ry', 'cz')
optimizer = SLSQP(maxiter=1000)

# intialize vqe instance
estimator = Estimator()
vqe = VQE(estimator, ansatz, optimizer)

result = vqe.compute_minimum_eigenvalue(ising_ham)
print(result)

# Adding the transpiler to reduce the circuit to QASM instructions
# supported by the backend
from qiskit import transpile 
from qiskit import Aer

opt_params=result.optimal_point
qc=ansatz.bind_parameters(opt_params)

backend = Aer.get_backend('aer_simulator')
# First we have to transpile the quantum circuit 
# to the low-level QASM instructions used by the 
# backend
qc_compiled = transpile(qc, backend)
qc_compiled.measure_active()
# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = backend.run(qc_compiled, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()
msrmnts = result_sim.get_counts()
print(msrmnts)

# since bits are flipped, the solution is
sol = qutils.get_most_likely(msrmnts=msrmnts)
energy = qutils.Ising_energy(assignment=sol, qubo=qubo)

print(sol, energy)

#qc.decompose().draw('mpl')

#ansatz.decompose().draw('mpl', style='iqx')

/var/folders/lg/jbb7l6y11_343nxvjxmyhh8h0000gn/T/ipykernel_91452/1181404923.py:13: DeprecationWarning: The property ``qiskit.utils.algorithm_globals.QiskitAlgorithmGlobals.random_seed`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. This algorithm utility has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. You can run ``pip install qiskit_algorithms`` and import ``from qiskit_algorithms.utils`` instead. 
  algorithm_globals.random_seed = 3


{   'aux_operators_evaluated': None,
    'cost_function_evals': 1481,
    'eigenvalue': -0.010722623470925705,
    'optimal_circuit': <qiskit.circuit.library.n_local.two_local.TwoLocal object at 0x11b50e770>,
    'optimal_parameters': {   ParameterVectorElement(θ[1]): -3.1509919654314507,
                              ParameterVectorElement(θ[2]): 3.1030733944772395,
                              ParameterVectorElement(θ[0]): -3.437370322329935,
                              ParameterVectorElement(θ[3]): -0.15426308168805547,
                              ParameterVectorElement(θ[4]): -6.203895134099096,
                              ParameterVectorElement(θ[5]): -1.4446227407930379,
                              ParameterVectorElement(θ[6]): 1.5718191966327526,
                              ParameterVectorElement(θ[7]): -4.938524304071353,
                              ParameterVectorElement(θ[8]): 3.1424801481409697,
                              ParameterVectorElement(θ[9]): -5.6015

/var/folders/lg/jbb7l6y11_343nxvjxmyhh8h0000gn/T/ipykernel_91452/1181404923.py:39: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.bind_parameters()`` is deprecated as of qiskit 0.45.0. It will be removed no earlier than 3 months after the release date. Use assign_parameters() instead
  qc=ansatz.bind_parameters(opt_params)


{'1110': 1, '1101': 2, '1011': 2, '0111': 1019}
[1, 0, 0, 0] -0.010743767178636635


In [6]:
# classical brute-force solution
qutils.find_optimum(qubo=qubo)

([1, 0, 0, 0], -0.010743767178636635)